In [1]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import scipy
from scipy.signal import convolve
from scipy import ndimage
import getBayer
% matplotlib inline
import io
import time
import copy
from numpy.lib.stride_tricks import as_strided

In [3]:
Im = getBayer.getBayer('pic2.jpeg')
bayer = getBayer.bayerGrid 
testIm = copy.deepcopy(Im)

# plt.imshow(Im/255)

In [7]:
GatR_B = np.array([[0,0,-1,0,0], [0,0,2,0,0],[-1,2,4,2,-1],[0,0,2,0,0],[0,0,-1,0,0]]) #Green at blue and red pixels
RBatG_row = np.array([[0,0,.5,0,0], [0,-1,0,-1,0],[-1,4,5,4,-1],[0,-1,0,-1,0],[0,0,.5,0,0]]) #Red at Green, in Red row, Blue column
RBatG_col = RBatG_row.T #Red at Green in Blue row, red coloumn. 
RBatBR = np.array([[0,0,-1.5,0,0], [0,2,0,2,0],[-1.5,0,6,0,-1.5],[0,2,0,2,0],[0,0,-1.5,0,0]])

RBatG_col

array([[ 0. ,  0. , -1. ,  0. ,  0. ],
       [ 0. , -1. ,  4. , -1. ,  0. ],
       [ 0.5,  0. ,  5. ,  0. ,  0.5],
       [ 0. , -1. ,  4. , -1. ,  0. ],
       [ 0. ,  0. , -1. ,  0. ,  0. ]])